# 3. Similitud entre productos

## Descripción
Un desafío constante en MELI es el de poder agrupar productos similares utilizando algunos atributos de estos como pueden ser el título, la descripción o su imagen. Para este desafío tenemos un dataset “items_titles.csv” que tiene títulos de 30 mil
productos de 3 categorías diferentes de Mercado Libre Brasil
## Entregable
El objetivo del desafío es poder generar una Jupyter notebook que determine cuán similares son dos títulos del dataset “item_titles_test.csv” generando como output un listado de la forma


| ITE_ITEM_TITLE | ITE_ITEM_TITLE | Score Similitud (0,1) |
|-----------|-----------|-----------|
| Zapatillas Nike   | Zapatillas Nike   | 1 |
| Zapatillas Nike   | Zapatillas Adidas | 0,5 |

donde ordenando por score de similitud podamos encontrar los pares de productosmás similares en nuestro dataset de test.

## Desarrollo del código

### 1. Intalación de la libreria de levenshtein
Esta libreria es necesaria para encontrar la similitud que existe ente los dos títulos comparados

In [1]:
#pip install python-levenshtein

In [2]:
#pip install nltk

In [3]:
#pip install spacy

Reiniciar el kernel para continuar

### 2. Importe de librerias necesarias para la transformación del dataset
- **pandas** para el manejo del dataset como dataframe.
- **re** para expresiones regulares, permite buscar y manipular patrones en cadenas de texto.
- **numpy** para cálculos numéricos y operaciones con matrices y arreglos.
- **ntlk** para NLP
- **Levenshtein** para encontrar el score
- **unidecode** para manejo de tildes
- **sklearn** para el modelo de entrenamiento sobre variables categóricas
- **spacy** para data cleaning de los valores

In [4]:
import pandas as pd
import re
import numpy as np
import nltk
import Levenshtein as lv
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import spacy
from spacy.lang.pt import Portuguese

# Descargar recursos adicionales para NLTK
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')

# Cargar modelo de lenguaje para el procesamiento de portugués en spaCy
nlp = Portuguese()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\valeria.mendez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\valeria.mendez\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\valeria.mendez\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\valeria.mendez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 3. Lectura del dataset
Este se encuentra contenido dentro de la misma ruta de este notebook por lo que se puede realizar la lectura directamente con el nombre del archivo

In [5]:
df_items_titles = pd.read_csv('items_titles.csv')

### 4. Limpieza de la base
Dentro del campo *ITE_ITEM_TITLE* se deben limpiar los valores para asegurar que la distancia de levenshtein sea la más fiel posible.

In [ ]:
# Función para eliminar *stop word* que hacen referencia a palabras conectores en portugues
def preprocess_text(text):
    # Tokenizar el texto en palabras
    words = nltk.word_tokenize(text)
    
    # Eliminar stop words
    stop_words = set(stopwords.words('portuguese'))
    words = [word for word in words if word.lower() not in stop_words]
    
    # Lematización
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Unir las palabras preprocesadas en un nuevo texto
    preprocessed_text = ' '.join(words)
    return preprocessed_text

# Función para eliminar las palabras conectores utilizando expresiones regulares
def quitar_conectores(texto):
    for conector in conectores_a_eliminar:
        texto = re.sub(r'\b' + re.escape(conector) + r'\b', '', texto, flags=re.IGNORECASE)
    return texto

# Función para quitar caracteres especiales utilizando expresiones regulares
def quitar_caracteres_especiales(texto):
    texto = re.sub(r'[^\w\s,]', '', texto)
    return texto

In [ ]:
# Utilizamos la función *def preprocess_text(text):* para quitar stop words
df_items_titles['Texto_Limpio'] = df_items_titles['ITE_ITEM_TITLE'].apply(preprocess_text)

# Eliminamos los espacios en blanco (espacios, tabulaciones, saltos de línea) del principio y final.
df_items_titles['Texto_Limpio'] = df_items_titles['Texto_Limpio'].str.strip()

# Colocamos el texto en minusculas
df_items_titles['Texto_Limpio'] = df_items_titles['Texto_Limpio'].str.lower()

# Aplicamos unidecode() para quitar los tildes de los caracteres acentuados en la columna 'columna'
df_items_titles['Texto_Limpio'] = df_items_titles['Texto_Limpio'].apply(lambda x: unidecode(x))

# Lista de palabras conectores a eliminar
conectores_a_eliminar = ['-',',']

# Aplicamos la función para quitar caracteres especiales y conectores
df_items_titles['Texto_Limpio'] = df_items_titles['Texto_Limpio'].apply(quitar_conectores)
df_items_titles['Texto_Limpio'] = df_items_titles['Texto_Limpio'].apply(quitar_caracteres_especiales)

# Termino la limpieza con la eliminación de las comas
df_items_titles['Texto_Limpio'] = df_items_titles['Texto_Limpio'].str.replace(',', '')

In [ ]:
# Vemos una pequeña muestra de la columna limpia con el titulo del item
print(df_items_titles.head())

### 5. Clasificación de categoría según el nombre del artículo
Esta es una primera clasificacion teniendo en cuenta palabras clave dentro del nombre del título del producto.

##### Exploración inicial
Realicé una exploración previa de las palabras con más frecuencia y posteriormente las coloque en tres categorías:
- Calzado: Incluye tenis, zapatillas, botas, marcas de zapatos, referencias de zapatos
- Bicicleta: Incluye pabras como bicicleta en inglés y español y marcas de productos de ciclismo
- TV: En donde se incluyen articulos relacionados con televisores, marcas de televisores y pantallas.

In [ ]:
#num_valores_unicos = df_items_otros['Texto_Limpio'].nunique()
# Crear una lista de todas las palabras en el DataFrame
todas_las_palabras = ' '.join(df_items_titles['Texto_Limpio']).split()

# Calcular la frecuencia de cada palabra
frecuencia_palabras = pd.Series(todas_las_palabras).value_counts()
print("cantidad de palabras: ", len(frecuencia_palabras))
print(frecuencia_palabras.head(30))

### 5.1. Categorización manual
Cree la función *categorizar_titulos(df, column):* para hacer una categorización manual de los productos de acuerdo a la primera exploración que vi de los valores que pueden contener las palabras y se frecuencia

In [ ]:
def categorizar_titulos(df, column):
    key_word_zapatos = ['tenis', 'sapatilha', 'sapato', 'sapatos', 'adidas', 'puma', 'nike', 'all star', 'air', 'slip', 'babuche', 'vans', 'tennis', 'sandalia', 'mizuno', 'shoes', 'zapatillas', 'calcado', 'calcados', 'bota', 'sapatenis', 'jordan', 'sneaker', 'new balance', 'botinha', 'chuteira']
    key_word_bicicleta = ['bike', 'bicicleta', 'specialized', 'bikes', 'bici', 'byke', 'motocicleta', 'oakley', 'bmx', 'shimano', 'disc']
    key_word_tv = ['tv', 'tela', 'televisao', 'televisor', 'samsung', 'panasonic', 'monitor']

    categorias = {
        'calzado': key_word_zapatos,
        'bicicleta': key_word_bicicleta,
        'tv': key_word_tv
    }

    # Inicializamos la columna 'Categoria' con valor 'otros'
    df['Categoria_Manual'] = 'otros'

    # Utilizamos un bucle para asignar la categoría adecuada a cada título
    for categoria, palabras_clave in categorias.items():
        for palabra in palabras_clave:
            df['Categoria_Manual'] = np.where(df[column].str.contains(palabra, case=False), categoria, df['Categoria_Manual'])

    return df

En las siguientes líneas validamos la distribución de los items con la clasificación manual por key words.

In [ ]:
# Uso de la función:
df_items_titles = categorizar_titulos(df_items_titles, 'Texto_Limpio')

# Calcular el conteo de cada variable en la columna 'respuestas'
conteo_variables = df_items_titles['Categoria_Manual'].value_counts()

# Calcular el porcentaje representativo de cada variable
porcentaje_representativo = conteo_variables / len(df_items_titles) * 100

print("Items:\n", conteo_variables, "\n", "Porcentajes representativos:\n", porcentaje_representativo)

### 5.2. Categorización a través de un modelo ML
Para optimizar más la clasificación manual, vamos a tomar la clasificación manual anterior como un conjunto de entrenamiento etiquetado para realizar el entrenamiento de modelo *MultinomialNB()* a partir de la vectorización de variables categóricas llamado *TfidfVectorizer()*, el cual consiste en darle un valor numérico a cada título de producto. De tal manera obtendremos una mejor salida de los tres grupos de clasificación existentes en el dataset

In [ ]:
# Dividir los datos para cada clasificación
df_calzado = df_items_titles[df_items_titles['Categoria_Manual'] == 'calzado']
df_tv = df_items_titles[df_items_titles['Categoria_Manual'] == 'tv']
df_bicicletas = df_items_titles[df_items_titles['Categoria_Manual'] == 'bicicleta']
df_otros = df_items_titles[df_items_titles['Categoria_Manual'] == 'otros']

# Seleccionar una cantidad similar de registros para cada clasificación
min_len = min(len(df_calzado), len(df_tv), len(df_bicicletas), len(df_otros))
df_calzado_sampled = df_calzado.sample(min_len, random_state=42)
df_tv_sampled = df_tv.sample(min_len, random_state=42)
df_bicicletas_sampled = df_bicicletas.sample(min_len, random_state=42)
df_otros_sampled = df_otros.sample(min_len, random_state=42)

# Unir los datos en un nuevo DataFrame balanceado
df_balanced = pd.concat([df_calzado_sampled, df_tv_sampled, df_bicicletas_sampled, df_otros_sampled], ignore_index=True)

# Dividir los datos en conjuntos de entrenamiento y test
X = df_balanced['Texto_Limpio']
y = df_balanced['Categoria_Manual']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### Vectorización de variables categóricas
A continuación hago uso de la función **TfidfVectorizer()** que me permite convertir los valores categoricos de *ITE_ITEM_TITLE*  en una matriz numérica que puede ser utilizada como entrada para el modelo de ML, en otras palabras toma el texto y lo convierte en valores numéricos a través de Tokenización, Construcción del vocabulario y Cálculo del TF-IDF que es la puntuación para cada palabra.

In [ ]:
# Vectorización del campo de ITE_ITEM_TITLE porque es una variable categórica
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

##### Escoger el modelo de entrenamiento

Escogí como modelo de clasificación **MultinomialNB()**, ya que crea un modelo simple de clasificación Naive Bayes multinomial. Esto se traduce en que el algoritmo Naive Bayes, al ser un clasificador probabilístico basado en el teorema de Bayes, toma independencia condicional entre las características de la variable de predicción y el valor de la variable objetivo/a predecir. Este modelo, adicionalmente tiene un componente de clasificación multinomial la cual se desenvuelve correctamente en problemas como este sobre análisis y clasificación de texto.

Algunas razones por las que funciona bien dentro del contexto de este reto:
- Clasificador Multinomial: MultinomialNB está diseñado para trabajar con características categóricas, como recuentos de palabras.
- Suavizado (Smoothing): MultinomialNB aplica suavizado (también conocido como corrección de Laplace) para evitar problemas de probabilidad cero cuando una palabra en un documento no aparece en los datos de entrenamiento.
- Vectorizer: Puedo utilizar las características obtenidas de la vectorización (la matriz TF-IDF)
- Evaluación: Puedo evaluar el rendimiento del modelo de maner sencilla como lo son metricas de accuracy, precision, recall, F1-score.

In [ ]:
#Entrenamiento del modelo
model = MultinomialNB()
model.fit(X_train_vectors, y_train)

In [ ]:
# Realizar predicciones y evaluar el modelo
y_pred = model.predict(X_test_vectors)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

classification_report_output = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_report_output)

Como se puede observar, tenemos un Accuracy mayor al 80% y considero que fue un modelo correctamente entrenado por las siguientes razones:
1. El Accuracy representa la proporción de predicciones correctas que hizo el modelo con respecto al total de muestras en el conjunto de datos.
2. Evaluación del Precision: Al tener un valor por encima del 70%, significa que el modelo está realizando correctamente sus predicciones en la mayoría de las muestras.
3. Overfitting: Un valor de Accuracy muy alto (cercano al 100%) puede indicar que el modelo está sobreajustando los datos de entrenamiento. Esto significa que el modelo se ha "memorizado" los datos de entrenamiento en lugar de aprender patrones generales, y por tanto no es capaz de enfrentarse a datos desconocidos y eso implicaria un rendimiento bajo a futuro.
4. Data Balanceada: Al tener una base de datos de entrenamiento balanceada en la cantidad de items por categoría, evito tener una categoría dominante, un modelo puede lograr un alto accuracy simplemente prediciendo siempre la clase dominante. Esto lo único que significa es que el modelo no logra hacer predicciones significativas para las categorías con una cantidad de items pequeña.

### 5.3. Clasificación del dataset original
Teniendo en cuenta que el modelo ya fue entrenado, tomare la columna de *Texto_limpio* del dataset y realizaré una nueva clasificación para refinar las categorías posibles

In [ ]:
# Obtener todos los valores de titulos de dataset
X_all = df_items_titles[df_items_titles['Categoria_Manual'] == 'otros']['Texto_Limpio']

# Vectorizar los títulos del dataset
X_all_vectors = vectorizer.transform(X_all)

# Predicción de la nueva categoría
y_pred_all = model.predict(X_all_vectors)

# Paso la columna 'Categoria_Manual' para mantener los valores originales
df_items_titles['Nueva_Clasificacion'] = df_items_titles['Categoria_Manual']

# Pasa los valores de la 'Categoria_Manual' con la 'Nueva_Clasificacion' 
df_items_titles.loc[df_items_titles['Categoria_Manual'] == 'otros', 'Nueva_Clasificacion'] = y_pred_all

# Elimino la columna de para quedarme unicamente con la columna que tiene el titulo limpio
df_items_titles = df_items_titles.drop('ITE_ITEM_TITLE', axis=1)

# Dejo como posibilidad guardar el nuevo dataframe con las predicciones de categorías
df_items_titles.to_csv('nuevo_dataframe_con_predicciones.csv', sep = '|', index=False)

##### Opcional
A continuación, evalúo que la categoría que cree de "otros" sea bastante pequeña para no ser tenida en cuenta. Al ser el 1,76% de la base de 30mil registros, no tomaré en cuenta estos valores ya que pueden haber valores basura o que necesitan de más características para ser clasificados en alguna de las 3 categorías.

In [ ]:
# Calcular el conteo de cada variable en la columna 'respuestas'
conteo_variables = df_items_titles['Nueva_Clasificacion'].value_counts()

# Calcular el porcentaje representativo de cada variable
porcentaje_representativo = conteo_variables / len(df_items_titles) * 100

print("Items:\n", conteo_variables, "\n", "Porcentajes representativos:\n", porcentaje_representativo)

### 6. Función para calcular el score de similitud
A continuación se encuentra la función que calcula la similitud entre dos titulos.
**Descripción de campos:**
- df: Corresponde al dataframe en el cual se va a evaluar la similitud.
- num_items_1: Corresponde al numero de items sobre los que se quiere investigar. Ejemplo: 1 equivale al primer titulo de todo el dataset, 2 a los dos primeros titulos de todo el dataset, etc...
- num_items_2: Corresponde al numero de items sobre los que se desea iterar en un mismo item. Ejemplo: Si coloca 10, va a tomar los 10 primeros items de todo el dataset para ser comparados contra la cantidad de items que colocó en num_items_1.
- titulos: Corresponde a la columna donde se encuentran los titulos de los productos.
- categoria_nombre: Corresponde al nombre de la columna que contiene la categoría que se desea evaluar.
- categoría_filtro: Corresponde a la categoría que desea filtrar. En este caso esa variable puede tomar los valores de: calzado, bicicleta o tv.

In [ ]:
def Calcular_Score_Similitud(df, num_items_1, num_items_2, titulos, categoria_nombre, categoria_filtro):
    # Filtrar el DataFrame por la categoría deseada
    df_items = df.loc[df[categoria_nombre] == categoria_filtro]

    # Crear un DataFrame para comparar textos
    df_item_compare = pd.DataFrame({'Texto_1': df_items[titulos].tolist(), 
                                          'Texto_2': df_items[titulos].tolist()})
    
    # Creación de lista para almacenar las distancias como score de similitud
    comparison_list = []
    
    # Creación de loop
    # Primer for indica el item fijo por el cual se va a evualuar
    for i in range(num_items_1):
        text1 = df_item_compare['Texto_1'][i]
        #Segundo for indica las veces que tiene que pasar el item del for anterior por cada uno de los items de este loop
        for j in range(num_items_2):
            text2 = df_item_compare['Texto_2'][j]
            # Obtener el score de similitud a partir de la distancia de levenshtein
            distance = lv.ratio(text1, text2)
            # Datos en forma de lista de listas
            datos = [df_item_compare['Texto_1'][i], df_item_compare['Texto_2'][j], distance]
            comparison_list.append(datos)

    # Nombres de las columnas para ser añadidos en un dataframe
    cols = ['Texto_1', 'Texto_2', 'Score Similitud (0,1)']

    # Crear el DataFrame utilizando pd.DataFrame()
    df_compared = pd.DataFrame(comparison_list, columns=cols)
    
    # Devolver el dataframe con los dos titulos comparados y el score
    return df_compared

### 7. Score de similitud para la categoría de calzado
En este caso estoy tomando los 3 primeros items de la categoría de calzado y los comparo con al menos 6 otras categorías

In [ ]:
# Llamada a la función con el DataFrame original y la categoría deseada
df_comparison_tenis = Calcular_Score_Similitud(df_items_titles, 3, 6, 'Texto_Limpio', 'Nueva_Clasificacion', 'calzado')
print(df_comparison_tenis)

### 8. Score de similitud para la categoría de bicicleta
En este caso estoy tomando los 3 primeros items de la categoría de bicicletas y los comparo con al menos 6 otras categorías

In [ ]:
# Llamada a la función con el DataFrame original y la categoría deseada
df_comparison_tenis = Calcular_Score_Similitud(df_items_titles, 3, 6, 'Texto_Limpio', 'Nueva_Clasificacion', 'bicicleta')
print(df_comparison_tenis)

### 9. Score de similitud para la categoría de Televisores
En este caso estoy tomando los 3 primeros items de la categoría de tv y los comparo con al menos 6 otras categorías

In [ ]:
# Llamada a la función con el DataFrame original y la categoría deseada
df_comparison_tenis = Calcular_Score_Similitud(df_items_titles, 2, 6, 'Texto_Limpio', 'Nueva_Clasificacion', 'tv')
print(df_comparison_tenis)